## ChromaDB を使った RAG のデモ

### OpenAI API Key とLLM、埋め込みモデルの設定

In [1]:
from dotenv import load_dotenv, find_dotenv
# 環境変数 OPENAI_API_KEY に OpenAI API Key を設定（.env ファイルの OPENAI_API_KEY からロード）
_ = load_dotenv(find_dotenv())
embedding_model = "text-embedding-3-large" # 埋め込みモデル
model = "gpt-4o" # LLM
temperature = 0 # LLMの生成のランダムさ

### LLMと埋め込みモデルの準備

In [2]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# LLMの準備
llm = ChatOpenAI(
    model=model, # モデル
    temperature=temperature, # ランダムさ
)

# 埋め込みモデルの準備
embeddings = OpenAIEmbeddings(model=embedding_model)

### 検索対象の ChromaDB コレクション名の設定
コレクションとは、ChromaDB におけるベクトルデータの保存単位で、RDB のテーブルのようなもの

In [3]:
#collection_name = "mycollection_100" # 検索対象のChromadb コレクション名（チャンクサイズ100文字で事前に作成済み）
collection_name = "mycollection_200" # 検索対象のChromadb コレクション名（チャンクサイズ200文字で事前に作成済み）
#collection_name = "mycollection_500" # 検索対象のChromadb コレクション名（チャンクサイズ500文字で事前に作成済み）
#collection_name = "mycollection_1000" # 検索対象のChromadb コレクション名（チャンクサイズ1000文字で事前に作成済み）

### VectorStoreの準備

In [4]:
import chromadb
from langchain_chroma import Chroma

# LangChain の Chromadb VectorStore のインスタンスを作成
vector_store = Chroma(
    client=chromadb.PersistentClient(path="./chroma_db"),
    collection_name=collection_name,
    embedding_function=embeddings,
    collection_metadata={"hnsw:space": "ip"} 
)

### Retrieverの準備

In [5]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)

### PromptTemplateの準備

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたの仕事は、human の question に答えることです。以下の手順で回答してください。前置きや思考経過は出力しないでください。context の中の source のファイル名には、page_content の主題が含まれています。最初に、 context に書かれている情報だけを使用して、question の質問に答えることができるかどうか判断してください、判断結果は出力しません。次に、context にある情報だけで回答が可能と判断した場合は、context に書かれている情報だけを使用して、question の質問に答えてください。context にある情報だけでは回答できないと判断した場合は、情報がないので回答できないと答えてください。\n\ncontext: {context}"),
        ("human", "question: {input}"),
    ]
)

### RAGチェーンの準備

In [7]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt_template
    | (lambda x: print(f"\n=== Generated Prompt ===\n{x}\n========================\n\n") or x)  # プロンプトを表示
    | llm
)

### 質問の定義

In [10]:
#query="ハグリッドがホグワーツへの入学案内書を持ってきたのはいつですか？それはどのような日でしたか？"
#query="アメリカの初代大統領は誰ですか？"
query="時空管理局のアースラの艦長は誰ですか？"
#query="フェイトが留学生として通った小学校は？"
#query="ユーノ・スクライアは、普段どのような動物の姿で過ごしていますか？"
#query="聖王のゆりかごとは何ですか、どのくらいの大きさですか？"
#query="スターライトブレイカーは誰のどのような魔法ですか？"
#query="高町なのはが小学生のときに通っていた学校は？"


### RAGの実行

In [11]:
response = rag_chain.invoke(query)
print(f"=== LLM response ======\n{response}\n========================\n")
print(f"=== 回答 ===============\n{response.content}\n========================\n")



=== Generated Prompt ===
messages=[SystemMessage(content="あなたの仕事は、human の question に答えることです。以下の手順で回答してください。前置きや思考経過は出力しないでください。context の中の source のファイル名には、page_content の主題が含まれています。最初に、 context に書かれている情報だけを使用して、question の質問に答えることができるかどうか判断してください、判断結果は出力しません。次に、context にある情報だけで回答が可能と判断した場合は、context に書かれている情報だけを使用して、question の質問に答えてください。context にある情報だけでは回答できないと判断した場合は、情報がないので回答できないと答えてください。\n\ncontext: [Document(metadata={'source': 'data\\\\Wikipedia-魔法科高校の劣等生.pdf'}, page_content='剛健でありながらスマートな雰囲気を⾝に纏う40歳前後の男性でリーナより2つ歳下の娘を持つ。スターズの中でも有数の常識⼈。2097年6⽉18⽇の早朝に、スターズ第⼗⼀隊のシャウラ少尉からの報告を受けてアルゴル少尉やミルファク少尉を含めた四⼈でリーナの救出に向かい、リーナを基地から脱出させることに成功するが、アルゴル少尉やシャウラ少尉と共に捕らえられる。'), Document(metadata={'source': 'data\\\\Wikipedia-八神はやて.pdf'}, page_content='⼀⽅フェイトを驚かせる⼀⾯もあった。『StrikerS』におけるはやて空港⽕災の後、管理局の対応の遅さを嘆き、周りの協⼒を得て本局遺失物管理部「機動六課」を4年がかりで設⽴[15]。機動六課の課⻑、機動六課本部隊舎の総部隊⻑、後⽅⽀援部隊「ロングアーチ」の指揮官を務める。'), Document(metadata={'source': 'data\\\\Wikipedia-魔法科高校の劣等生.pdf'}, page_content='⼆⼈はこれを受け⼊れる。そして⼆⼈を乗せた⾼千穂は深雪とリーナによる疑似瞬間移動によ